## Load canonical vertices

In [1]:
import numpy as np

In [2]:
with open('.././data/v05.txt', 'r') as f:
    L = f.readlines()
L=[l.strip() for l in L];
L = [l for l in L if "N" not in l] #...select the matrices only
L = L[:-1]
L = [l.split() for l in L]         #...use delimiter
L = [list(map(int, l)) for l in L] #...convert to int


In [3]:
L = np.reshape(L, (-1,4, 5))
L[0].T

array([[ 1,  0,  0,  0],
       [ 0,  1,  0,  0],
       [ 0,  0,  1,  0],
       [ 1,  2,  3,  5],
       [-2, -3, -4, -5]])

In [4]:
len(L)

1561

## Get the plücker coords
We need to input matrix form into this

In [5]:
def plk(vec):
    vert = matrix(vec);
    ker=vert.kernel().matrix()
    return ker.minors(ker.nrows())

In [6]:
print(plk(((1,0,0,0),(0,1,0,0),(0,0,1,0),(1,2,3,5),(-2,-3,-4,-5))))

[1, 1, 1, 1, 1]


In [7]:
plklist=[];
for i in range(0,len(L)):
    plklist.append(plk(L[i].T.tolist()))

In [8]:
plklist[1]

[1, 1, 1, 1, 1]

In [9]:
L[0:5]

array([[[ 1,  0,  0,  1, -2],
        [ 0,  1,  0,  2, -3],
        [ 0,  0,  1,  3, -4],
        [ 0,  0,  0,  5, -5]],

       [[ 1,  0,  2,  3, -6],
        [ 0,  1,  4,  3, -8],
        [ 0,  0,  5,  0, -5],
        [ 0,  0,  0,  5, -5]],

       [[ 1,  0,  0,  0, -1],
        [ 0,  1,  0,  0, -1],
        [ 0,  0,  1,  0, -1],
        [ 0,  0,  0,  1, -1]],

       [[ 1,  1,  1,  1, -4],
        [ 0,  5,  0,  0, -5],
        [ 0,  0,  5,  0, -5],
        [ 0,  0,  0,  5, -5]],

       [[ 1,  0,  0,  0, -2],
        [ 0,  1,  0,  0, -1],
        [ 0,  0,  1,  0, -1],
        [ 0,  0,  0,  1, -1]]])

## Compute the Polytope Volume

In [10]:
def polyvol(vec):
    vert = matrix(vec);
    vol=Polyhedron(vertices=vert).volume()
    return vol

In [11]:
volist=[];
for i in range(0,len(L)):
    volist.append(24*polyvol(L[i].T))

# Save polytope data
We could save the data as strings (with literals) in the database.

In [12]:
for k in volist:
    print(k)

25
125
5
625
6
648
10
1000
8
1024
27
243
12
432
8
512
16
512
14
686
12
864
18
1944
32
256
18
216
12
324
12
384
15
375
32
512
16
512
18
648
20
500
24
576
12
1152
24
576
24
1728
16
256
16
256
24
432
24
432
24
432
9
729
18
486
27
243
16
1024
32
512
30
1350
32
128
25
125
24
216
24
288
20
320
32
256
12
576
12
648
24
432
32
512
216
36
36
972
36
1296
30
1800
24
144
24
192
12
288
18
216
24
144
24
384
40
400
24
576
24
768
48
384
20
800
40
400
42
1176
24
162
24
216
36
108
32
256
27
243
16
256
24
192
45
225
32
128
48
192
30
360
40
320
250
40
48
288
18
486
40
400
36
324
162
54
30
600
24
576
48
288
15
675
30
450
48
576
45
225
48
768
48
864
20
1000
42
882
48
864
36
1944
36
144
24
192
48
144
36
216
32
128
216
48
24
384
32
512
64
256
56
392
42
504
54
648
32
512
28
784
56
392
60
900
24
1152
48
1152
48
576
36
108
32
128
36
216
64
128
24
192
48
288
36
144
48
96
32
256
24
324
48
216
64
256
108
72
48
384
48
288
36
384
72
192
24
432
60
300
54
648
30
750
66
726
48
864
48
2304
32
128
48
192
48
216
72
144
27
2

### Create Database

In [345]:
import sqlite3

con = sqlite3.connect("dim_4_plucker.db")
cur = con.cursor()

cur.execute("CREATE TABLE poly_vol_4(vertices, plucker, volume, num_vertices)")
con.commit()

In [356]:
for i, j, k in zip(L, plklist, volist):
    cur.execute("INSERT INTO poly_vol_4 VALUES (?, ?, ?, ?)", ("{}".format(i.tolist()), "{}".format(j), float(k), "{}".format(len(j))))
con.commit()

### Compute Polytope Reflexivity

In [316]:
from ast import literal_eval
literal_eval(t[1][0])

[[1, 0, 2, 3, -6], [0, 1, 4, 3, -8], [0, 0, 5, 0, -5], [0, 0, 0, 5, -5]]